In [9]:
import os
import numpy as np
import cantera as ct
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
dfs = []

for i in range(1, 9):

    base_path = 'flame_speed_ch4_20210531_102003'
    file_path = f'/RUN_{i:06}/CKSoln_Flame_SpeedC1.ckcsv'
    path = base_path + file_path
    
    df_read = pd.read_csv(path, skiprows=1, header=None)

    columns_long = df_read.iloc[:, 0] + df_read.iloc[:, 1] 

    columns = []
    for name in columns_long:
        columns.append(name.split('_Run')[0])

    df = pd.DataFrame(data=df_read.iloc[:, 2:].T.to_numpy(), columns=columns)
    
    dfs.append(df)

In [4]:
print(columns)

['Distance', 'Pressure', 'Mass_density', 'Axial_velocity', 'Cross-section_area', 'Net_heat_production_from_gas-phase_reactions', 'EINO', 'EINO2', 'EINOX', 'EICO', 'Unburned_fuel_fraction', 'Temperature', 'Mole_fraction_H2', 'Mole_fraction_H', 'Mole_fraction_O', 'Mole_fraction_O2', 'Mole_fraction_OH', 'Mole_fraction_H2O', 'Mole_fraction_HO2', 'Mole_fraction_H2O2', 'Mole_fraction_C', 'Mole_fraction_CH', 'Mole_fraction_CH2', 'Mole_fraction_CH2(S)', 'Mole_fraction_CH3', 'Mole_fraction_CH4', 'Mole_fraction_CO', 'Mole_fraction_CO2', 'Mole_fraction_HCO', 'Mole_fraction_CH2O', 'Mole_fraction_CH2OH', 'Mole_fraction_CH3O', 'Mole_fraction_CH3OH', 'Mole_fraction_C2H', 'Mole_fraction_C2H2', 'Mole_fraction_C2H3', 'Mole_fraction_C2H4', 'Mole_fraction_C2H5', 'Mole_fraction_C2H6', 'Mole_fraction_HCCO', 'Mole_fraction_CH2CO', 'Mole_fraction_HCCOH', 'Mole_fraction_N', 'Mole_fraction_NH', 'Mole_fraction_NH2', 'Mole_fraction_NH3', 'Mole_fraction_NNH', 'Mole_fraction_NO', 'Mole_fraction_NO2', 'Mole_fraction

# Pre-procces
get Mass Fraction

In [5]:
# Load Molecular Weight
df_species = pd.read_csv('Species_List_GRI.csv')
columns_X = 'Mole_fraction_' + df_species['Name'].to_numpy()
columns_Y = 'Mass_fraction_' + df_species['Name'].to_numpy()
molecular_weights = df_species['Molecular_Weight'].to_numpy()

dfs_Y = []

for df in dfs:

    # Prepare ndarray from dataframe
    mole_fractions = df[columns_X].to_numpy()
    mean_molecular_weight = df['Molecular_weight'].to_numpy()

    # Calculate mass fraction
    mass_fractions = np.divide(np.multiply(mole_fractions, molecular_weights).T, mean_molecular_weight).T
    df_Y = pd.DataFrame(mass_fractions, columns=columns_Y)

    dfs_Y.append(df_Y)

# Test to get binary diffusion coefficient
using cantera

In [12]:
T = df['Temperature'][0]
P = df['Pressure'][0]
X = df[columns_X].to_numpy()[0, :]

In [14]:
gas = ct.Solution('gri30.yaml')
gas.TPX = T, P, X

In [23]:
gas.binary_diff_coeffs.shape

(53, 53)

# Calculate Diffusion Coefficient

In [ ]:
D_dash = D - mu*Y*E/dYdx